# Modified/Optimized copy of Sept 25th version of the script:
https://colab.research.google.com/drive/13zYtgUggPKa0aPVWNTLgckn3F8sz5dd1#scrollTo=608iiBHu8gbb

## Report of Main changes summary from Sept 25 - Oct 4:


Here’s a summary of the key changes from the original script you provided on Sep 25 to the current working script:

### 1. Refactor of `aggregate_monthly_degree_days` Function
**Original Version:**
- The aggregate_monthly_degree_days function worked with individual (year, month) arguments and handled region boundaries.

**Updated Version:**
- The updated script now works by unpacking a (year, month) tuple inside aggregate_monthly_degree_days, and region handling has been removed.

**Benefit**:
- allows for a more streamlined approach when processing a list of (year, month) pairs,
- simplifying the code by focusing only on the data processing itself rather than additional region constraints.



### 2. Handling 4-Hourly Degree Days Calculation
**Original Version:**
- The 4-hourly aggregation logic was not explicitly managed.

**Updated Version:**
- The compute_4hourly_degree_days function was streamlined to process 4-hourly intervals, and the date properties (like hour and day_of_year) are added directly using the add_date_properties function.





### 3. Use of Python Tuples for (year, month) and ee.List in generate_monthly_aggregates
**Original Version:**
- The original approach involved passing year and month as individual parameters to various functions.

**Updated Version**:
- The script now passes monthly dates as a list of tuples (i.e., [(2023, 1), (2023, 2)]) to the generate_monthly_aggregates function, and converts them into Earth Engine’s ee.List to work within GEE’s ecosystem.

**Benefit:**
- This allows the code to process all monthly data in a more concise and efficient way using Earth Engine’s .map() function, which results in more efficient and scalable processing for multiple months and years.



### 4. Efficient Mapping and Image Collection Creation
**Original Version:**
- The script processed individual months and years in loops and used filters for regional boundaries.

**Updated Version:**
- The new generate_monthly_aggregates function processes a list of dates using Earth Engine’s .map() functionality, which reduces the overhead of nested loops and simplifies the handling of multiple months and years.

**Benefit**: It allows the processing to be handled more efficiently within Earth Engine’s framework and is more scalable for large datasets.



### 5. Zonal Statistics Handling Remained Unchanged (SAME)
- The zonal_stats function was carried over from the original script without modification, as the original zonal statistics implementation was robust and compatible with the new data structures.
- The key functionality (applying reducers to feature collections) remained intact.


### 6. Optimized Export Process
**Original Version** : The export process was somewhat repetitive, with individual year-month combinations being handled manually.

**Updated Version**: The export process now iterates through all monthly_dates generated by the updated generate_monthly_aggregates function, ensuring that the correct data is passed to the zonal statistics function and exported efficiently.

**Benefit**: This change allows for faster and more flexible export processes, handling multiple months and years in one sweep, reducing manual setup for each export job.


### 7. Added Flexibility in Base Temperatures
**Original Version**: The base temperature handling was more rigid.

**Updated Version**: The generate_monthly_aggregates function now allows for multiple base temperatures, processing each temperature threshold through a list, and combining the results in a single image collection.

 **Benefit** : This allows the user to compute degree days across a range of temperature thresholds in one run.



### Summary of the Key Changes:
- (year, month) Tuple Handling: Refactor to pass monthly dates as tuples and process them in a more efficient way.
- 4-Hourly Degree Day Calculation: Proper handling of 4-hour intervals, including adding date properties and computing daily averages to account for missing data.
- Efficient .map() Usage: Replacing loops with Earth Engine’s .map() to handle all monthly dates efficiently.
- Streamlined Export Process: Simplified the export logic to iterate over months and years using zonal statistics.
- Flexible Base Temperature Handling: Added the ability to compute degree days across a range of temperatures.

#START

In [11]:
PROJECT = "treegrowth-471820"

# Authorization and Imports

In [12]:
#!gcloud auth login --project {PROJECT}


In [13]:
import datetime
import ee
import math


In [14]:
ee.Authenticate()
ee.Initialize(project=PROJECT)

Function to filter 4-hourly intervals and add day properties to each image

# Computing Degree-Day Values from 4-Hourly Data

1. Helps track and group temperature data into 4-hour intervals.

In [15]:
def add_date_properties(image):
    date = ee.Date(image.get('system:time_start'))
    hour = date.get('hour')
    day_of_year = date.getRelative('day', 'year')
    return image.set('day_of_year', day_of_year).set('hour', hour)

2. Calculates degree days relative to the base temperature (e.g., -20°C).

In [16]:
def compute_4hourly_degree_days(image, base_temp=-20):
    temp_band = image.select('temperature_2m').subtract(273.15)  # Convert Kelvin to Celsius
    degree_days = temp_band.subtract(base_temp).max(0)
    return degree_days.rename(f'degree_days_above_{base_temp}')

3. For each year and month, the function filters ERA5 hourly
temperature data.

- It applies the 4-hourly degree days calculation.

- Degree days are aggregated by day and converted into monthly sums.

In [17]:
def aggregate_monthly_degree_days(monthly_date, base_temp):
    year, month = monthly_date  # unpacking a tuple
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')

    era_hourly = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filterDate(start_date, end_date)

    # compute 4-hourly degree days + add date properties
    degree_days = era_hourly.map(lambda img: compute_4hourly_degree_days(img, base_temp)).map(add_date_properties)

    # sum degree days by day
    daily_sum = degree_days.reduce(ee.Reducer.sum())
    # count the number of 4-hour intervals per day
    daily_count = degree_days.reduce(ee.Reducer.count())
    # calculate the average degree days per day to account for missing data
    daily_avg = daily_sum.divide(daily_count).rename(f'daily_avg_degree_days_above_{base_temp}')

    # sum the daily averages over the month
    monthly_sum = daily_avg.reduce(ee.Reducer.sum()).rename(f'monthly_sum_degree_days_above_{base_temp}')

    # add year and month as bands
    year_band = ee.Image.constant(year).rename('year').toFloat()
    month_band = ee.Image.constant(month).rename('month').toFloat()

    # Combine results
    combined = monthly_sum.addBands(year_band).addBands(month_band)

    return combined

4. Aggregates temperature data (max, min, mean, sum) for each month.
Adjusts for Celsius conversion if the temperature is the band.

In [18]:
def aggregate_monthly_data(year, month, band_name, operation):
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')
    era_hourly = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filterDate(start_date, end_date).select(band_name)

    if operation == 'max':
        aggregated_data = era_hourly.max()
    elif operation == 'min':
        aggregated_data = era_hourly.min()
    elif operation == 'mean':
        aggregated_data = era_hourly.mean()
    elif operation == 'sum':
        aggregated_data = era_hourly.sum()
    else:
        raise ValueError(f"Unsupported operation: {operation}")

    # Convert temperature from Kelvin to Celsius if the band name includes 'temperature'
    if 'temperature' in band_name:
        aggregated_data = aggregated_data.subtract(273.15)

    return aggregated_data.set('system:time_start', start_date.millis())


5. This function loops through all the months of the specified years.

It computes maximum, minimum, and mean temperature values as well as precipitation.
Degree day values for multiple base temperatures are calculated and concatenated into a single image.

In [19]:
def generate_monthly_aggregates(monthly_dates, base_temps):
    def process_monthly_date(monthly_date):
        year = ee.Number(monthly_date.get(0))
        month = ee.Number(monthly_date.get(1))

        monthly_max = aggregate_monthly_data(year, month, 'temperature_2m', 'max').rename('max_temp')
        monthly_min = aggregate_monthly_data(year, month, 'temperature_2m', 'min').rename('min_temp')
        monthly_mean = aggregate_monthly_data(year, month, 'temperature_2m', 'mean').rename('mean_temp')
        monthly_precip = aggregate_monthly_data(year, month, 'total_precipitation', 'sum').rename('total_precip')

        degree_days_list = [aggregate_monthly_degree_days((year, month), base_temp).select(f'monthly_sum_degree_days_above_{base_temp}')
                            for base_temp in base_temps]
        degree_days_combined = ee.Image.cat(degree_days_list)

        # Add year and month as bands
        year_band = ee.Image.constant(year).rename('year').toFloat()
        month_band = ee.Image.constant(month).rename('month').toFloat()

        # Combine everything
        combined = monthly_max.addBands(monthly_min).addBands(monthly_mean).addBands(monthly_precip).addBands(degree_days_combined).addBands(year_band).addBands(month_band)
        return combined

    # Convert monthly dates to an ee.List -> Python list of tuples into an ee.List of ee.Lists
    ee_monthly_dates = ee.List([ee.List([ee.Number(year), ee.Number(month)]) for year, month in monthly_dates])

    # Apply the process for each monthly date using .map()
    monthly_aggregates = ee.ImageCollection.fromImages(
        ee_monthly_dates.map(lambda monthly_date: process_monthly_date(ee.List(monthly_date)))
    )

    return monthly_aggregates

In [20]:
#coordinates = ee.FeatureCollection("projects/era5-land-project/assets/itrdb_locations_unique_with_duplicate_lat_lon_info")
# Load Canada sites
import pandas as pd 
CSV_PATH      = "ring_width_output_new/tree_ring_coordinates.csv"
sites_df = pd.read_csv(CSV_PATH)
sites_df = sites_df.dropna(subset=["lat","lon"]).copy()
sites_df["site_id"] = sites_df.get("site_id", pd.Series([f"site_{i:06d}" for i in range(len(sites_df))]))
print(f"Loaded {len(sites_df)} sites")

def _row_to_feat(r):
    geom = ee.Geometry.Point([float(r["lon"]), float(r["lat"])])
    return ee.Feature(geom, {"site_id": str(r["site_id"])})
coordinates = ee.FeatureCollection([_row_to_feat(r) for _, r in sites_df.iterrows()])

Loaded 598 sites


## Testing for a smaller range of data

In [21]:
monthly_aggregates_ic = generate_monthly_aggregates([(2023, 1), (2023, 2)], list(range(-5, 0)))

print(monthly_aggregates_ic.size().getInfo())
print(monthly_aggregates_ic.first().bandNames().getInfo())



2
['max_temp', 'min_temp', 'mean_temp', 'total_precip', 'monthly_sum_degree_days_above_-5', 'monthly_sum_degree_days_above_-4', 'monthly_sum_degree_days_above_-3', 'monthly_sum_degree_days_above_-2', 'monthly_sum_degree_days_above_-1', 'year', 'month']


# Export piece

In [22]:
def zonal_stats(ic, fc, params=None):
    _params = {
        'reducer': ee.Reducer.mean(),  # Change this as needed (e.g., sum, median)
        'scale': 1000,  # change to 9km?
        'crs': None,
        'bands': None,
        'bandsRename': None,
        'imgProps': None,
        'imgPropsRename': None,
        'datetimeName': 'datetime',
        'datetimeFormat': 'YYYY-MM-dd HH:mm:ss'
    }
    if params:
        for param in params:
            _params[param] = params[param] or _params[param]

    img_rep = ic.first()
    non_system_img_props = ee.Feature(None).copyProperties(img_rep).propertyNames()
    if not _params['bands']:
        _params['bands'] = img_rep.bandNames()
    if not _params['bandsRename']:
        _params['bandsRename'] = _params['bands']
    if not _params['imgProps']:
        _params['imgProps'] = non_system_img_props
    if not _params['imgPropsRename']:
        _params['imgPropsRename'] = _params['imgProps']

    def map_function(img):
        img = ee.Image(img.select(_params['bands'], _params['bandsRename']))
        img = img.set(_params['datetimeName'], img.date().format(_params['datetimeFormat']))
        img = img.set('timestamp', img.get('system:time_start'))
        props_from = ee.List(_params['imgProps']).cat(ee.List([_params['datetimeName'], 'timestamp']))
        props_to = ee.List(_params['imgPropsRename']).cat(ee.List([_params['datetimeName'], 'timestamp']))
        img_props = img.toDictionary(props_from).rename(props_from, props_to)
        fc_sub = fc.filterBounds(img.geometry())
        return img.reduceRegions(
            collection=fc_sub,
            reducer=_params['reducer'],
            scale=_params['scale'],
            crs=_params['crs']
        ).map(lambda f: f.set(img_props))

    results = ic.map(map_function).flatten()
    return results

In [23]:
def export_era5_data_by_year(years, base_temps, feature_collection, export_folder):
    # Create a list of (year, month) tuples
    monthly_dates = [(year, month) for year in years for month in range(1, 13)]  # List of (year, month)

    monthly_aggregates_ic = generate_monthly_aggregates(monthly_dates, base_temps)

    # Apply zonal statistics to the monthly weather aggregates (image collection)
    params = {'reducer': ee.Reducer.mean(), 'scale': 1000} # change to 9km?
    ptsERA5monthly = zonal_stats(monthly_aggregates_ic, feature_collection, params)

    # Define the export parameters
    export_description = f"monthly_aggregates_export_degree_days_{years[0]}-{years[-1]}"
    export_params = {
        'collection': ptsERA5monthly,
        'description': export_description,
        'folder': export_folder,
        'fileFormat': 'CSV'
        }

    # Start the export task
    task = ee.batch.Export.table.toDrive(**export_params)
    task.start()
    print(f"Exporting data: {export_description} to Google Drive in folder {export_folder}")

In [24]:
base_temps = list(range(-20, 36))
export_folder = "export_era5_monthly_20250428"

In [ ]:
start_year = 1995
end_year = 2005
chunk_size = 3 

for year in range(start_year, end_year, chunk_size):
  years = list(range(year, min(year + chunk_size, end_year)))
  print(f"Processing years: {years}")
  export_era5_data_by_year(years, base_temps, coordinates, export_folder)

Processing years: [1995, 1996, 1997]
Exporting data: monthly_aggregates_export_degree_days_1995-1997 to Google Drive in folder export_era5_monthly_20250428
Processing years: [1998, 1999, 2000]
Exporting data: monthly_aggregates_export_degree_days_1998-2000 to Google Drive in folder export_era5_monthly_20250428
Processing years: [2001, 2002, 2003]
Exporting data: monthly_aggregates_export_degree_days_2001-2003 to Google Drive in folder export_era5_monthly_20250428
Processing years: [2004]
Exporting data: monthly_aggregates_export_degree_days_2004-2004 to Google Drive in folder export_era5_monthly_20250428


In [26]:
years = list(range(2001, 2004)) # list(range(2022,2024))  # Ideal list of chunks: (1950, 1975), (1975, 2000), (2000, 2024)
print(years)
export_era5_data_by_year(years, base_temps, coordinates, export_folder)

[2001, 2002, 2003]
Exporting data: monthly_aggregates_export_degree_days_2001-2003 to Google Drive in folder export_era5_monthly_20250428
